### Multiple Knapsack Problem

Exact solution from ChatGPT

In [1]:
using Combinatorics

In [2]:
function knapsack_exact(weights, values, capacities)
    n = length(weights)
    m = length(capacities)
    best_value = 0
    best_assignment = []

    for k in 0:n
        for subset in combinations(1:n, k)
            for assignment in product(fill(0:m, k)...)
                valid = true
                knapsack_weights = zeros(Int, m)

                for (item, knapsack) in zip(subset, assignment)
                    if knapsack > 0
                        knapsack_weights[knapsack] += weights[item]
                        if knapsack_weights[knapsack] > capacities[knapsack]
                            valid = false
                            break
                        end
                    end
                end

                if valid
                    value = sum([knapsack > 0 ? values[item] : 0 for (item, knapsack) in zip(subset, assignment)])
                    if value > best_value
                        best_value = value
                        best_assignment = [(i, a) for (i, a) in zip(subset, assignment) if a > 0]
                    end
                end
            end
        end
    end

    return best_value, best_assignment
end

knapsack_exact (generic function with 1 method)

In [3]:
weights = [2, 3, 4, 5, 6]
values = [3, 4, 5, 8, 10]
capacities = [10, 12]

println(knapsack_exact(weights, values, capacities))

UndefVarError: UndefVarError: product not defined

Heuristic from ChatGPT: Greedy

In [4]:
function knapsack_heuristic(weights, values, capacities)
    n = length(weights)
    m = length(capacities)
    ratio = values ./ weights
    indices = sortperm(ratio, rev=true)
    assignment = zeros(Int, n)
    knapsack_weights = zeros(Int, m)

    for i in indices
        for k in 1:m
            if knapsack_weights[k] + weights[i] <= capacities[k]
                assignment[i] = k
                knapsack_weights[k] += weights[i]
                break
            end
        end
    end

    total_value = sum([assignment[i] > 0 ? values[i] : 0 for i in 1:n])
    assigned_items = [(i, assignment[i]) for i in 1:n if assignment[i] > 0]

    return total_value, assigned_items
end


knapsack_heuristic (generic function with 1 method)

In [5]:
weights = [2, 3, 4, 5, 6]
values = [3, 4, 5, 8, 10]
capacities = [10, 12]

println(knapsack_heuristic(weights, values, capacities))

(30, [(1, 1), (2, 2), (3, 2), (4, 2), (5, 1)])


Branch&Bound from ChatGPT

In [10]:
using JuMP
using Gurobi

# Define the problem data (number of knapsacks, items, capacities, weights, and values)
num_knapsacks = 3
num_items = 4
capacities = [10, 15, 20]
weights = [4, 6, 3, 9]
values = [8, 10, 6, 12]

# Create a function to solve a relaxation of the MKP
function solve_relaxation(weights, values, capacities)
    model = Model(Gurobi.Optimizer)  # Adjust the solver and solver options
    set_time_limit_sec(model, 60)
    # Define variables
    @variable(model, x[1:num_items] >= 0, Bin)
    
    # Objective: Maximize the total value
    @objective(model, Max, sum(x[i] * values[i] for i in 1:num_items))
    
    # Constraints: Knapsack capacity constraints
    for k in 1:num_knapsacks
        @constraint(model, sum(x[i] * weights[i] for i in 1:num_items) <= capacities[k])
    end
    
    # Solve the relaxation
    optimize!(model)
    
    return objective_value(model), value.(x)
end

# Create a function for the Branch and Bound algorithm
function branch_and_bound(capacities, weights, values)
    best_solution = -Inf
    best_x = nothing

    open_nodes = [(0, 0, zeros(Int, num_items))]
    
    while !isempty(open_nodes)
        node = popfirst!(open_nodes)
        upper_bound, level, x = node
        
        # Solve the relaxation for the current node
        relaxation_value, relaxation_x = solve_relaxation(weights, values, capacities)
        
        if relaxation_value <= best_solution
            continue
        end
        
        if level == num_items
            best_solution = relaxation_value
            best_x = relaxation_x
        else
            # Branch: Two child nodes (0 and 1)
            for branch in 0:1
                new_x = copy(x)
                new_x[level + 1] = branch
                new_upper_bound = upper_bound + branch * values[level + 1]

                # Check if the node is feasible
                feasible = true
                for k in 1:num_knapsacks
                    if sum(new_x[i] * weights[i] for i in 1:num_items) > capacities[k]
                        feasible = false
                        break
                    end
                end

                if feasible
                    push!(open_nodes, (new_upper_bound, level + 1, new_x))
                end
            end
        end
    end

    return best_solution, best_x
end

best_solution, best_x = branch_and_bound(capacities, weights, values)
println("Best Solution Value: $best_solution")
println("Best Solution (0/1 vector): $best_x")


Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-03
Set parameter TimeLimit to value 60
Set parameter TimeLimit to value 60
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 4 columns and 12 nonzeros
Model fingerprint: 0x87db38a3
Variable types: 0 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [3e+00, 9e+00]
  Objective range  [6e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+01]
Found heuristic solution: objective 18.0000000
Presolve removed 3 rows and 4 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 18 

Optimal solution

Best Solution Value: 18.0


Best Solution (0/1 vector): [1.0, 1.0, 0.0, 0.0]


In [11]:
using JuMP
using Gurobi

# Define the problem data (number of knapsacks, items, capacities, weights, and values)
num_knapsacks = 3
num_items = 4
capacities = [10, 15, 20]
weights = [4, 6, 3, 9]
values = [8, 10, 6, 12]

# Create a master problem with an empty set of columns
master = Model(Gurobi.Optimizer)

# Create variables for the master problem
@variable(master, z[1:num_items] >= 0, Bin)  # Binary variables representing columns

# Objective: Maximize the reduced cost of each column
@objective(master, Max, sum(reduced_cost[i] * z[i] for i in 1:num_items))

# Create a subproblem to solve the knapsack relaxation
function solve_subproblem(master, z)
    subproblem = Model(Gurobi.Optimizer)

    # Variables: y[j] represents the dual value of the j-th knapsack constraint
    @variable(subproblem, y[1:num_knapsacks] >= 0)

    # Constraints: Knapsack relaxation constraints
    @constraint(subproblem, [k in 1:num_knapsacks], sum(z[i] * weights[i] for i in 1:num_items) <= capacities[k])

    # Objective: Minimize the sum of dual values (for pricing)
    @objective(subproblem, Min, sum(y[k] for k in 1:num_knapsacks))

    optimize!(subproblem)

    # Get the dual values (y) for knapsack constraints
    dual_values = JuMP.value.(y)

    return objective_value(subproblem), dual_values
end

# Define the column generation loop
while true
    # Solve the subproblem to obtain a new column (item) with reduced cost
    reduced_cost, dual_values = solve_subproblem(master, z)
    
    if reduced_cost <= 0.0
        break  # No more improving columns
    end

    # Create and add a new column to the master problem
    @variable(master, z_new >= 0, Bin)
    @constraint(master, [k in 1:num_knapsacks], sum(z_new * weights[i] for i in 1:num_items) <= capacities[k])
    
    # Solve the master problem to update the solution with the new column
    optimize!(master)
end

# Retrieve the final solution from the master problem
solution = value.(z)

println("Optimal Solution (0/1 vector): $solution")


Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-03


MethodError: MethodError: no method matching getindex(::typeof(reduced_cost), ::Int64)

In [12]:
reduced_cost

reduced_cost (generic function with 1 method)